<a href="https://colab.research.google.com/github/NVIDIA/synthda/blob/main/colab/brev_demo_generateSynthDa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, output
drive.mount('/content/drive')


## Downloading and Setting up SynthDa repo (Optional if you have your own models you'd like to use instead )

In [ ]:
!mkdir -p /content/drive/MyDrive/autosynthda/indiv


!git clone https://github.com/NVIDIA/synthda /content/drive/MyDrive/autosynthda/indiv

## Install Requirements for each new runtime

In [ ]:
%cd /content/drive/MyDrive/autosynthda/indiv
!ls

In [ ]:
!pip install -r /content/drive/MyDrive/autosynthda/indiv/synthda/components/requirements.txt

In [ ]:
!pip install numpy==1.23.5
!pip install yacs
!pip install filterpy
!pip install smplx==0.1.28
!pip install trimesh==3.9.0
!pip install chumpy==0.70
!pip install dotenv

In [ ]:
# hardcoded fix for chumpy for python 3.11

import inspect

# Monkey patch getargspec for chumpy compatibility with Python ≥3.11
if not hasattr(inspect, 'getargspec'):
    from collections import namedtuple

    ArgSpec = namedtuple('ArgSpec', 'args varargs keywords defaults')

    def getargspec(func):
        sig = inspect.signature(func)
        args = []
        varargs = None
        keywords = None
        defaults = []

        for name, param in sig.parameters.items():
            if param.kind in (param.POSITIONAL_ONLY, param.POSITIONAL_OR_KEYWORD):
                args.append(name)
                if param.default is not param.empty:
                    defaults.append(param.default)
            elif param.kind == param.VAR_POSITIONAL:
                varargs = name
            elif param.kind == param.VAR_KEYWORD:
                keywords = name

        return ArgSpec(args, varargs, keywords, tuple(defaults) if defaults else None)

    inspect.getargspec = getargspec


## Download and Install Repos [If done previously, then can skip this]

In [ ]:
!git clone https://github.com/Vegetebird/StridedTransformer-Pose3D.git \
             /content/drive/MyDrive/autosynthda/indiv/StridedTransformer-Pose3D

!git clone https://github.com/EricGuo5513/text-to-motion.git \
             /content/drive/MyDrive/autosynthda/indiv/text-to-motion

!git clone https://github.com/wangsen1312/joints2smpl.git \
             /content/drive/MyDrive/autosynthda/indiv/joints2smpl

%cd /content/drive/MyDrive/autosynthda/indiv

# Download Blender into the current directory
!wget -P . https://download.blender.org/release/Blender3.0/blender-3.0.0-linux-x64.tar.xz

# Extract in-place
!tar -xf blender-3.0.0-linux-x64.tar.xz -C .

#!git clone https://github.com/facebookresearch/SlowFast.git /content/drive/MyDrive/autosynthda/indiv

In [ ]:
import os

# switch into the Drive-based repo folder
%cd /content/drive/MyDrive/autosynthda/indiv/StridedTransformer-Pose3D

# make sure the pretrained checkpoint dir exists
!mkdir -p checkpoint/pretrained

# download the two refine/non-refine checkpoints if missing
if not os.path.exists('checkpoint/pretrained/refine_4365.pth'):
    !gdown https://drive.google.com/uc?id=1aDLu0SB9JnPYZOOzQsJMV9zEIHg2Uro7 \
            -O checkpoint/pretrained/refine_4365.pth
if not os.path.exists('checkpoint/pretrained/no_refine_4365.pth'):
    !gdown https://drive.google.com/uc?id=1l63AI9BsNovpfTAbfAkySo9X2MOWgYZH \
            -O checkpoint/pretrained/no_refine_4365.pth

# ensure the demo lib checkpoint dir exists
!mkdir -p demo/lib/checkpoint

# download YOLOv3 weights
if not os.path.exists('demo/lib/checkpoint/yolov3.weights'):
    !gdown https://drive.google.com/uc?id=1gWZl1VrlLZKBf0Pfkj4hKiFxe8sHP-1C \
            -O demo/lib/checkpoint/yolov3.weights

# download HRNet pose model
if not os.path.exists('demo/lib/checkpoint/pose_hrnet_w48_384x288.pth'):
    !gdown https://drive.google.com/uc?id=1CpyZiUIUlEjiql4rILwdBT4666S72Oq4 \
            -O demo/lib/checkpoint/pose_hrnet_w48_384x288.pth


In [ ]:
import os

# switch into the Drive-based repo folder
%cd /content/drive/MyDrive/autosynthda/indiv/text-to-motion

# make sure the checkpoints1 dir exists
!mkdir -p checkpoints1

# download the model checkpoint if missing
if not os.path.exists('checkpoints1/checkpoints'):
    !gdown https://drive.google.com/uc?id=12liZW5iyvoybXD8eOw4VanTgsMtynCuU \
            -O checkpoints1/checkpoints


In [ ]:
# 2. Switch into the text-to-motion folder on Drive
%cd /content/drive/MyDrive/autosynthda/indiv/text-to-motion

# 3. Ensure the checkpoints1 dir exists
!mkdir -p checkpoints1

# 4a. Download & unzip the first model
if not os.path.exists('checkpoints1/model.zip'):
    !gdown --id 12liZW5iyvoybXD8eOw4VanTgsMtynCuU -O checkpoints1/model.zip
!unzip -q checkpoints1/model.zip -d checkpoints1

# 4b. Download & unzip the second model
if not os.path.exists('checkpoints1/model2.zip'):
    !gdown --id 1IgrFCnxeg4olBtURUHimzS03ZI0df_6W -O checkpoints1/model2.zip
!unzip -q checkpoints1/model2.zip -d checkpoints1


## Run Each Component for Sanity Check

In [ ]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install iopath fvcore pytorchvideo tensorboard setuptools torchinfo opencv-python seaborn numpy Pillow scikit-learn


In [ ]:
# Check for the version of torch and whether cuda is used
import torch
print("Torch file:", torch.__file__)
print("CUDA attr exists:", hasattr(torch, "cuda"))

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())

In [ ]:
env_vars = """
STRIDED_TRANSFORMER_PATH=/content/drive/MyDrive/autosynthda/indiv/StridedTransformer-Pose3D
TEXT_TO_MOTION_PATH=/content/drive/MyDrive/autosynthda/indiv/text-to-motion
JOINTS2SMPL_PATH=/content/drive/MyDrive/autosynthda/indiv/joints2smpl
SLOWFAST_PATH=/content/drive/MyDrive/autosynthda/indiv/SlowFast
BLENDER_BIN=/content/drive/MyDrive/autosynthda/indiv/blender-3.0.0-linux-x64/blender
BLENDER_ROOT=/content/drive/MyDrive/autosynthda/indiv/blender-3.0.0-linux-x64
BLENDER_PATH=/content/drive/MyDrive/autosynthda/indiv/blender-3.0.0-linux-x64/blender
"""
with open("/content/drive/MyDrive/autosynthda/indiv/synthda/components/.env", "w") as f:
    f.write(env_vars.strip())


In [ ]:
## Note: You should replace the files with the edited versions to make it work in Colab. It can be found in the colab/ folder on the Github

In [ ]:
# switch into the Drive-based StridedTransformer folder
%cd /content/drive/MyDrive/autosynthda/indiv/StridedTransformer-Pose3D

# run the visualization on your sample video (adjust the path if your video lives elsewhere)
!python demo/vis.py --video sample_video.mp4


In [ ]:
# switch into the Drive-mounted text-to-motion folder
%cd /content/drive/MyDrive/autosynthda/indiv/text-to-motion

# generate motions using your input.txt in that folder
!python gen_motion_script.py \
    --name Comp_v6_KLD01 \
    --text_file input.txt \
    --repeat_time 1


In [ ]:
import importlib
import smplx
importlib.reload(smplx)

# switch into the Drive-mounted joints2smpl folder
%cd /content/drive/MyDrive/autosynthda/indiv/joints2smpl

# list the SMPL model files in your Drive repo
!ls -lh /content/drive/MyDrive/autosynthda/indiv/joints2smpl/smpl_models/smpl/

# run the fitting script on your test_motion2.npy (make sure the .npy is in this folder or give a full path)
!python fit_seq.py --files test_motion2.npy


In [ ]:
import shutil
import os

# Source folder in your Drive-mounted joints2smpl repo
src = "/content/drive/MyDrive/autosynthda/indiv/joints2smpl/demo/demo_results/test_motion2"

# Destination folder in your Drive-mounted SynthDA repo
dst = "/content/drive/MyDrive/autosynthda/indiv/synthda/components/renders/test_motion2"

# Ensure the destination parent directory exists
os.makedirs(os.path.dirname(dst), exist_ok=True)

# If it already exists, remove the old one
if os.path.exists(dst):
    shutil.rmtree(dst)

# Copy the entire demo_results folder into SynthDA's renders directory
shutil.copytree(src, dst)
print("✅ Folder copied to expected Blender input location.")

# Verify
expected_path = dst
print("Exists:", os.path.exists(expected_path))
print("Contents:", os.listdir(expected_path) if os.path.exists(expected_path) else "Path does not exist.")
!ls /content/drive/MyDrive/autosynthda/indiv/synthda/components/renders/test_motion2

In [10]:
#!/content/blender-3.0.0-linux-x64/blender -b -P /content/synthda/components/animation_pose.py -- --name <folder_with_ply_files>

# may not work on on Colab immediately, as running blender headlessly on Colab has some challenges. However you will be able to download the .fbx as well to view the animation locally.
# For Colab, the suggested alternative is to use pyrender instead, which we use in the demo function created for BREV below!

# Step into the folder that contains the script AND the renders/ directory
# %cd /content/drive/MyDrive/autosynthda/indiv/synthda/components

# Now launch Blender relative to this directory
#!/content/drive/MyDrive/autosynthda/indiv/blender-3.0.0-linux-x64/blender \
#    -b \
#    -P animation_pose.py \
#    -- --name test_motion2


## Generating Your Own Demo Data [Trying out Real-Mix from SynthDa]

In [ ]:
!pip install trimesh pyrender imageio[ffmpeg] pyglet
!pip install av imageio[ffmpeg] trimesh pyrender pyglet
!pip install python-dotenv


In [ ]:
import numpy as np
from scipy.interpolate import interp1d
from pathlib import Path
import sys
sys.path.append("/content/drive/MyDrive/autosynthda/indiv/synthda/components")

from optimisation.optimisation_utils import map_h36m_to_smpl, upsample_pose_data, compute_P_opt

# real_path_npz_1 refers to the First Generated Tracked-Motion Data
# real_path_npz_2 refers to the Second Generated Tracked-Motion Data

# used when one is real and another is real too
def main_real_real(real_path_npz_1, real_path_npz_2, folder_path):

    # folder_path_variations = folder_path + "all_variations/"
    # print(folder_path_variations)
    folder_path_variations = Path(folder_path) / "all_variations"
    folder_path_variations.mkdir(parents=True, exist_ok=True)

    # converting npz to npy
    real_data_1_path = folder_path + '/output_keypoints_3d_real1.npy'
    real_data_2_path = folder_path + '/output_keypoints_3d_real2.npy'

    print(real_data_1_path)
    print(real_data_2_path)

    # mapping the npz to npy file with 22 joints
    real_data_1 = map_h36m_to_smpl(real_path_npz_1)
    np.save(real_data_1_path, real_data_1)

    # mapping the npz to npy file with 22 joints
    real_data_2 = map_h36m_to_smpl(real_path_npz_2)
    np.save(real_data_2_path, real_data_2)

    # checking which frames are shorter, to match the frames to be the same
    # Load the real and synthetic pose data from .npy files
    P_r = np.load(real_data_1_path)  # Shape: (J, 3)
    P_s = np.load(real_data_2_path)  # Shape: (J, 3)

    # Determine the smaller array
    if P_r.shape[0] > P_s.shape[0]:
        smaller_array = real_data_2_path
        bigger_array = real_data_1_path
        max_frames = P_r.shape[0]
    else:
        smaller_array = real_data_1_path
        bigger_array = real_data_2_path
        max_frames = P_s.shape[0]

    # upsampling the smaller array to the number of frames
    pose_data_upsampled = upsample_pose_data(smaller_array, target_frames=max_frames)

    extended_new_path = smaller_array.replace(".npy", "_extended.npy")

    # saving the new array
    np.save(extended_new_path, pose_data_upsampled)
    print(pose_data_upsampled.shape)

    # generating the all variations of optimisation files
    weight_pairs = [
        (0.1, 0.9),
        (0.2, 0.8),
        (0.3, 0.7),
        (0.4, 0.6),
        (0.5, 0.5),
        (0.6, 0.4),
        (0.7, 0.3),
        (0.8, 0.2),
        (0.9, 0.1)
    ]

    if bigger_array == real_data_1_path:
        for w_A, w_B in weight_pairs:
            P_opt = compute_P_opt(real_data_1_path, extended_new_path, alpha=0.5, w_A=w_A, w_B=w_B)
            np.save(f"{folder_path_variations}/_euclidean_distances_wA{w_A}_wB{w_B}.npy", P_opt)

    # in a situation where the real_path is shorter than the synthetic path
    if smaller_array == real_data_1_path:
        for w_A, w_B in weight_pairs:
            P_opt = compute_P_opt(real_data_2_path, extended_new_path, alpha=0.5, w_A=w_A, w_B=w_B)
            np.save(f"{folder_path_variations}/_euclidean_distances_wA{w_A}_wB{w_B}.npy", P_opt)


In [ ]:
import os
# try EGL first:
os.environ["PYOPENGL_PLATFORM"] = "egl"
# if that still complains you can try:
# os.environ["PYOPENGL_PLATFORM"] = "osmesa"

from pathlib import Path
import math
import trimesh
import pyrender
from PIL import Image

# ---- adjust these paths ----
ply_dir = Path("/content/drive/MyDrive/autosynthda/indiv/synthda/components/renders/test_video_3_test_video_1_euclidean_distances_wA0.5_wB0.5")
out_dir = Path("/content/drive/MyDrive/autosynthda/indiv/synthda/components/renders/testpng_video3_gen_frames")
# -----------------------------

out_dir.mkdir(exist_ok=True, parents=True)

# build a single scene/camera/light
scene = pyrender.Scene()
camera = pyrender.PerspectiveCamera(yfov=math.radians(50.0))
light  = pyrender.DirectionalLight(color=[1.0,1.0,1.0], intensity=2.0)
scene.add(camera, pose=trimesh.transformations.translation_matrix([0,0,2]))
scene.add(light,  pose=trimesh.transformations.translation_matrix([0,0,2]))

# now the offscreen renderer
r = pyrender.OffscreenRenderer(viewport_width=1920,
                               viewport_height=1080,
                               point_size=1.0)

files = sorted(ply_dir.glob("*.ply"))
for i, ply in enumerate(files):
    mesh = trimesh.load_mesh(ply)
    # if your models need upright correction:
    # mesh.apply_transform(trimesh.transformations.rotation_matrix(
    #     math.radians(90), [1,0,0]))
    m = pyrender.Mesh.from_trimesh(mesh, smooth=False)

    # swap out old mesh
    for node in list(scene.mesh_nodes):
        scene.remove_node(node)
    scene.add(m)

    color, _ = r.render(scene)
    Image.fromarray(color).save(out_dir / f"frame_{i:04d}.png")
    if i % 10 == 0:
        print(f"Rendered {i+1}/{len(files)} frames")

r.delete()
print("✅ All frames saved in", out_dir)


In [ ]:
import os
import shutil
import subprocess
import random
import numpy as np
from pathlib import Path
from itertools import combinations
from dotenv import dotenv_values

import math
import trimesh
import pyrender
from PIL import Image

# ─── Config ────────────────────────────────────────────────────────────────────
env = dotenv_values("/content/drive/MyDrive/autosynthda/indiv/synthda/components/.env")
STRIDED     = Path(env["STRIDED_TRANSFORMER_PATH"])
TEXT2M      = Path(env["TEXT_TO_MOTION_PATH"])
JOINTS2SMPL = Path(env["JOINTS2SMPL_PATH"])
COMP_ROOT   = Path("/content/drive/MyDrive/autosynthda/indiv/synthda/components")
ANIM_REND   = COMP_ROOT/"renders"  # base for saving ply→png sequences

# optimisation utils
from optimisation.optimisation_utils import map_h36m_to_smpl, upsample_pose_data, compute_P_opt

# ─── Helper: headless PLY→PNG via pyrender ──────────────────────────────────────
def render_ply_sequence(ply_folder: Path, png_out: Path,
                        width=1920, height=1080, fps=27):
    # set EGL for headless
    os.environ["PYOPENGL_PLATFORM"] = "egl"

    png_out.mkdir(parents=True, exist_ok=True)
    scene = pyrender.Scene()
    cam = pyrender.PerspectiveCamera(yfov=math.radians(50.0))
    light = pyrender.DirectionalLight(color=[1.0,1.0,1.0], intensity=2.0)
    scene.add(cam, pose=trimesh.transformations.translation_matrix([0,0,2]))
    scene.add(light,  pose=trimesh.transformations.translation_matrix([0,0,2]))
    r = pyrender.OffscreenRenderer(viewport_width=width,
                                   viewport_height=height,
                                   point_size=1.0)

    ply_files = sorted(ply_folder.glob("*.ply"))
    for idx, ply in enumerate(ply_files):
        mesh = trimesh.load_mesh(str(ply))
        # if you need the 90° upright correction, uncomment:
        # mesh.apply_transform(trimesh.transformations.rotation_matrix(
        #     math.radians(90), [1,0,0]))
        m = pyrender.Mesh.from_trimesh(mesh, smooth=False)
        # clear old mesh nodes
        for node in list(scene.mesh_nodes):
            scene.remove_node(node)
        scene.add(m)
        color, _ = r.render(scene)
        Image.fromarray(color).save(png_out/f"frame_{idx:04d}.png")
    r.delete()

# ─── Main pipeline ────────────────────────────────────────────────────────────
def both_real_main(*, weight_A, input_dir, output_dir, num_pairs):
    random.seed(42)
    video_dir      = Path(input_dir)
    out_root       = Path(output_dir)
    video_gen_root = video_dir.parent/f"videos_generated_real2_{weight_A}"
    video_gen_root.mkdir(parents=True, exist_ok=True)

    vids = list(video_dir.glob("*.mp4"))
    pairs = list(combinations(vids, 2))
    selection = random.sample(pairs, min(num_pairs, len(pairs)))

    for v1_path, v2_path in selection:
        v1, v2      = v1_path.name, v2_path.name
        pair_name   = v1_path.stem + "_" + v2_path.stem
        pair_folder = out_root / pair_name
        pair_folder.mkdir(parents=True, exist_ok=True)

        # copy originals
        shutil.copy(v1_path, pair_folder/v1)
        shutil.copy(v2_path, pair_folder/v2)

        # 1) StridedTransformer
        for vn in (v1, v2):
            tgt = STRIDED/"demo"/"video"/vn
            tgt.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy(pair_folder/vn, tgt)
            subprocess.run(
                ["python","demo/vis.py","--video",vn],
                cwd=STRIDED, check=True
            )

        # 2) map NPZ → SMPL .npy
        npz1 = STRIDED/"demo"/"output"/v1_path.stem/"output_3D"/"output_keypoints_3d.npz"
        npz2 = STRIDED/"demo"/"output"/v2_path.stem/"output_3D"/"output_keypoints_3d.npz"
        out1 = pair_folder/"output_keypoints_3d_real1.npy"
        out2 = pair_folder/"output_keypoints_3d_real2.npy"
        p1 = map_h36m_to_smpl(str(npz1)); np.save(out1, p1)
        p2 = map_h36m_to_smpl(str(npz2)); np.save(out2, p2)

        # align lengths
        if p1.shape[0] < p2.shape[0]:
            p1 = upsample_pose_data(str(out1), p2.shape[0]); np.save(out1,p1)
        else:
            p2 = upsample_pose_data(str(out2), p1.shape[0]); np.save(out2,p2)

        # 3) generate mixtures
        var_folder = pair_folder/"all_variations"
        var_folder.mkdir(exist_ok=True, parents=True)
        for wA in [round(x,1) for x in np.linspace(0.1,0.9,9)]:
            wB = round(1-wA,1)
            Popt = compute_P_opt(str(out1), str(out2),
                                  alpha=0.5, w_A=wA, w_B=wB)
            np.save(var_folder/f"euclidean_distances_wA{wA}_wB{wB}.npy", Popt)

        # pick the requested mix
        target_npy = var_folder/f"euclidean_distances_wA{weight_A}_wB{round(1-weight_A,1)}.npy"
        if not target_npy.exists():
            raise FileNotFoundError(f"Missing variation: {target_npy}")

        # 4) joints2smpl → .ply frames
        dest_npy = JOINTS2SMPL/"demo"/"demo_data"/(pair_name+"_"+target_npy.name)
        dest_npy.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(target_npy, dest_npy)
        subprocess.run(
            ["python","fit_seq.py","--files",dest_npy.name,"--num_smplify_iters","1"],
            cwd=JOINTS2SMPL, check=True
        )

        # find the .ply output folder
        ply_src = JOINTS2SMPL/"demo"/"demo_results"/dest_npy.stem
        if not ply_src.exists():
            raise FileNotFoundError(f"joints2smpl output not found: {ply_src}")

        # 5) render PLY → PNG
        png_folder = ANIM_REND/f"{pair_name}_{target_npy.stem}"
        render_ply_sequence(ply_src, png_folder)

        # 6) ffmpeg assemble
        out_mp4 = video_gen_root/f"{pair_name}.mp4"
        subprocess.run([
            "ffmpeg","-y",
            "-framerate", str(27),
            "-i", str(png_folder/"frame_%04d.png"),
            "-c:v","libx264","-pix_fmt","yuv420p",
            str(out_mp4)
        ], check=True)

        print("✅ Generated:", out_mp4)


In [ ]:
both_real_main(
  weight_A=0.2,
  input_dir="/content/drive/MyDrive/autosynthda/indiv/synthda/components/dataset/specific_data",
  output_dir="/content/drive/MyDrive/autosynthda/indiv/synthda/components/dataset/data_manipulation",
  num_pairs=1
)
